# Agent-based application of Large Language Models

Large Language Model (LLM) agents can expand the capabilities of LLMs beyond single call completions. They are crucial for creating dynamic, interactive, and context-aware applications.   
In this notebook, we build the following agents:
- An LLM agent that calls a single python interpreter tool.
- A ReAct based LLM agent that uses two built-in tools, a calculator and a wikipedia search.
- User defined agent and its tools.

In this notebook, we build an LLM agent that works based on the [ReAct](https://react-lm.github.io/) prompting (Action + Observation + Thought).   
The Agent uses two built-in tools. One is a calculator API and the other is a wikipedia search agent. The agent decides which tool to use based on the input question (prompt).



In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

import pprint
# A function for printing nicely
def nprint(text, indent=2):
    pp = pprint.PrettyPrinter(indent=indent)
    pp.pprint(text)

## Parameters:

In [2]:
modelID = "gpt-3.5-turbo"

Defining the llm:

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model=modelID)

## Agent with the Python Interpreter

In [4]:
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool
from langchain_experimental.agents.agent_toolkits import create_python_agent
python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

agent = create_python_agent(
    llm,
    tool=repl_tool,
    verbose=True
)

In [5]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [6]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

d:\Portfolio\LLMs Applications\.llmenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

Python REPL can execute arbitrary code. Use with caution.


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] [2.73s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "We can use the sorted() function in Python to sort the list of customers based on last name first and then first name.\nAction: python_repl\nAction Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "We can use the sorted() function in Python to sort the list of customers based on last name first and then first name.\nAction: pyth

## ReAct Agent with multiple built-in tools
Defining the tools:
- llm-math: a calculator API
- wikipedia: a wikipedia search

In [7]:
from langchain.agents import load_tools
tools = load_tools(["llm-math","wikipedia"], llm=llm)
for tool in tools:
    print(f"{tool.name}: {tool.description}")

Calculator: Useful for when you need to answer questions about math.
wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.


We define the llm agent based on the [ReAct](https://react-lm.github.io/) prompting (Action + Observation + Thought).
The agent decides which tool to use based on the input question (prompt).

In [8]:
from langchain.agents import initialize_agent, AgentType

agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

d:\Portfolio\LLMs Applications\.llmenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


A detailed execution of the agent:

In [9]:
import langchain
langchain.debug = True
agent("What is the 25% of 300?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the 25% of 300?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the 25% of 300?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\n

d:\Portfolio\LLMs Applications\.llmenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] [1.18s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: We can use the calculator tool to find 25% of 300.\nAction:\n```\n{\n  \"action\": \"Calculator\",\n  \"action_input\": \"25% of 300\"\n}\n```",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Thought: We can use the calculator tool to find 25% of 300.\nAction:\n```\n{\n  \"action\": \"Calculator\",\n  \"action_input\": \"25% of 300\"\n}\n```",
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 43,
                "prompt_tokens": 315,
           

{'input': 'What is the 25% of 300?', 'output': '75.0'}

### A Wikipedia related prompt

In [10]:
completion = agent('What is the definition of Artificial Intelligence?')

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the definition of Artificial Intelligence?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the definition of Artificial Intelligence?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_i

In [11]:
nprint(completion['output'])

('Artificial intelligence (AI) is intelligence exhibited by machines, '
 'particularly computer systems, that enables machines to perceive their '
 'environment, learn, and take actions to achieve defined goals. It is a field '
 'of research in computer science that has seen significant advancements, '
 'particularly in recent years with the development of deep learning and the '
 'transformer architecture.')


## Agent with multiple user-defined tools:
Now, we create user-defined tools and a tool calling agent to that selects between them to work as a scientific calculator.   

Multiplication tool:

In [12]:
from langchain_core.tools import tool

@tool
def multiply(first_number: float, second_number: float) -> float:
    """Multiply two numbers."""
    return first_number * second_number

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'first_number': {'title': 'First Number', 'type': 'number'}, 'second_number': {'title': 'Second Number', 'type': 'number'}}


Build a couple of more tools:

In [13]:
@tool
def add(first_number: float, second_number: float) -> float:
    "Add two numbers."
    return first_number + second_number

@tool
def natural_log(number: float) -> float:
    "natural logarithm of number."
    import math
    return math.log(number)    


@tool
def exponentiate(base: float, exponent: float) -> float:
    "Exponentiate the base to the exponent power."
    return base**exponent

@tool
def sine(degrees: int) -> int:
    "Returns the sine of the given angle in degrees."
    import math
    radians = math.radians(degrees)
    return math.sin(radians)    


tools = [multiply, add, exponentiate, natural_log, sine]

In [14]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


Constructing the tool calling agent and its executor:

In [15]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[Syste

Creating the agent executor:

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b

Now test the calculator agent with the following sample prompts:

In [17]:
langchain.debug = False
agent_executor.invoke(
    {
        # "input": "multiply 2.5 by 4.5"
        # "input": "What is the natural logarithm of 4?"
        # "input": "What is the sine of 90?"
        "input": "calculate this mathematical expression: sin(ln(5^3 * 5 + 10))."
        # "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 5, 'exponent': 3}`


125.0
Invoking: `multiply` with `{'first_number': 125, 'second_number': 5}`


625.0
Invoking: `add` with `{'first_number': 625, 'second_number': 10}`


635.0
Invoking: `natural_log` with `{'number': 635}`


6.453624998892692
Invoking: `sine` with `{'degrees': 6}`


0.10452846326765347The value of \( \sin(\ln(5^3 \times 5 + 10)) \) is approximately 0.1045.

> Finished chain.


{'input': 'calculate this mathematical expression: sin(ln(5^3 * 5 + 10)).',
 'output': 'The value of \\( \\sin(\\ln(5^3 \\times 5 + 10)) \\) is approximately 0.1045.'}